In [ ]:
import whitebox
import rasterio
import numpy as np
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'gdal'

In [2]:
wbt = whitebox.WhiteboxTools()
wbt.set_working_dir("D:\Master\DigitalTwins\data")

### Remove Depressions

In [55]:
input_dem = r"merged_dtm_2022.tif"
wbt.verbose = True
wbt.compress_rasters = False

In [70]:
# Preprocess DEM
filled_dem =r"filled_dem.tif"
wbt.fill_depressions(
        dem=input_dem,
        output=filled_dem,
        fix_flats=True,
        flat_increment=None,
        max_depth=None
    )

.\whitebox_tools.exe --run="FillDepressions" --wd="D:\Master\DigitalTwins\data" --dem='merged_dtm_2022.tif' --output='filled_dem.tif' --fix_flats -v --compress_rasters=False

******************************
* Welcome to FillDepressions *
* Powered by WhiteboxTools   *
* www.whiteboxgeo.com        *
******************************


Reading data...
Finding pit cells: 12%
Finding pit cells: 25%
Finding pit cells: 37%
Finding pit cells: 50%
Finding pit cells: 62%
Finding pit cells: 75%
Finding pit cells: 87%
Finding pit cells: 100%
Filling depressions: 0%
Filling depressions: 1%
Filling depressions: 2%
Filling depressions: 3%
Filling depressions: 4%
Filling depressions: 5%
Filling depressions: 6%
Filling depressions: 7%
Filling depressions: 8%
Filling depressions: 9%
Filling depressions: 10%
Filling depressions: 11%
Filling depressions: 12%
Filling depressions: 13%
Filling depressions: 14%
Filling depressions: 15%
Filling depressions: 16%
Filling depressions: 17%
Filling depressions: 18%
Filling depressions: 19%
Filling depressions: 20%
Filling depressions: 21%
Filling depressions: 22%
Filling depressions: 23%
Filling depressions: 24%
Filling depressions: 25%
Filling depressions: 26%
Filling depressions: 27%
Filling depressions: 28%
Filling depressions: 29%
Filling depressions: 30%
Filling depressions: 31%
Filling d

0

In [ ]:
# Calculate flow direction
output_pointer = r"d8_pointer.tif"
wbt.d8_pointer(
        dem=filled_dem,
        output=output_pointer,
        esri_pntr=False 
    )

.\whitebox_tools.exe --run="D8Pointer" --wd="D:\Master\DigitalTwins\data" --dem='filled_dem.tif' --output='d8_pointer.tif' -v --compress_rasters=False

****************************


* Welcome to D8Pointer     *
* Powered by WhiteboxTools *
* www.whiteboxgeo.com      *
****************************
Reading data...
Progress: 0%
Progress: 1%
Progress: 2%
Progress: 3%
Progress: 4%
Progress: 5%
Progress: 6%
Progress: 7%
Progress: 8%
Progress: 9%
Progress: 10%
Progress: 11%
Progress: 12%
Progress: 13%
Progress: 14%
Progress: 15%
Progress: 16%
Progress: 17%
Progress: 18%
Progress: 19%
Progress: 20%
Progress: 21%
Progress: 22%
Progress: 23%
Progress: 24%
Progress: 25%
Progress: 26%
Progress: 27%
Progress: 28%
Progress: 29%
Progress: 30%
Progress: 31%
Progress: 32%
Progress: 33%
Progress: 34%
Progress: 35%
Progress: 36%
Progress: 37%
Progress: 38%
Progress: 39%
Progress: 40%
Progress: 41%
Progress: 42%
Progress: 43%
Progress: 44%
Progress: 45%
Progress: 46%
Progress: 47%
Progress: 48%
Progress: 49%
Progress: 50%
Progress: 51%
Progress: 52%
Progress: 53%
Progress: 54%
Progress: 55%
Progress: 56%
Progress: 57%
Progress: 58%
Progress: 59%
Progress: 60%
Progress: 61%
Progress: 

0

In [69]:
# Calculate flow accumulation
flow_accum = "flow_accumulation.tif"
wbt.d8_flow_accumulation(
        i=filled_dem,
        output=flow_accum,
        out_type="specific contributing area",  # or "cells" or "catchment area"
        log=False,  # Set True for log-transformed output
        clip=False,
        pntr=False,  # Input is DEM, not pointer
        esri_pntr=False
    )

.\whitebox_tools.exe --run="D8FlowAccumulation" --wd="D:\Master\DigitalTwins\data" --input='filled_dem.tif' --output='flow_accumulation.tif' --out_type=specific contributing area -v --compress_rasters=False

*********************************
* Welcome to D8FlowAccumulation *
* Powered by WhiteboxTools      *
* www.whiteboxgeo.com           *
*********************************


Reading data...
Flow directions: 0%
Flow directions: 1%
Flow directions: 2%
Flow directions: 3%
Flow directions: 4%
Flow directions: 5%
Flow directions: 6%
Flow directions: 7%
Flow directions: 8%
Flow directions: 9%
Flow directions: 10%
Flow directions: 11%
Flow directions: 12%
Flow directions: 13%
Flow directions: 14%
Flow directions: 15%
Flow directions: 16%
Flow directions: 17%
Flow directions: 18%
Flow directions: 19%
Flow directions: 20%
Flow directions: 21%
Flow directions: 22%
Flow directions: 23%
Flow directions: 24%
Flow directions: 25%
Flow directions: 26%
Flow directions: 27%
Flow directions: 28%
Flow directions: 29%
Flow directions: 30%
Flow directions: 31%
Flow directions: 32%
Flow directions: 33%
Flow directions: 34%
Flow directions: 35%
Flow directions: 36%
Flow directions: 37%
Flow directions: 38%
Flow directions: 39%
Flow directions: 40%
Flow directions: 41%
Flow directions: 42%
Flow directions: 43%
Flow directions: 44%
Flow directions: 45%
Flow directions: 46%
Flow di

0

### WATERSHED DELINEATION

In [12]:
def delineate_watershed(d8_pointer, pour_points, output_watershed):
    """
    Delineate watershed from pour points (outlets).
    """
    wbt.watershed(
        d8_pntr=d8_pointer,
        pour_pts=pour_points,
        output=output_watershed,
        esri_pntr=False
    )
    print(f"Watershed saved to: {output_watershed}")


In [13]:
def delineate_all_basins(d8_pointer, output_basins):
    """
    Delineate all drainage basins to edge.
    """
    wbt.basins(
        d8_pntr=d8_pointer,
        output=output_basins,
        esri_pntr=False
    )
    print(f"Basins saved to: {output_basins}")

In [14]:
def delineate_subbasins(d8_pointer, streams, output_subbasins):
    """
    Delineate subbasins for each stream link.
    """
    wbt.subbasins(
        d8_pntr=d8_pointer,
        streams=streams,
        output=output_subbasins,
        esri_pntr=False
    )
    print(f"Subbasins saved to: {output_subbasins}")

### TOPOGRAPHIC INDICES

In [15]:
def calculate_elevation_above_stream(dem, streams, output):
    """
    Calculate height above nearest drainage (HAND).
    Useful for flood mapping.
    """
    wbt.elevation_above_stream(
        dem=dem,
        streams=streams,
        output=output
    )
    print(f"Elevation above stream saved to: {output}")


In [16]:
def calculate_downslope_distance_to_stream(dem, streams, output):
    """
    Calculate distance to stream along flow path.
    """
    wbt.downslope_distance_to_stream(
        dem=dem,
        streams=streams,
        output=output,
        dinf=False  # Use D8; set True for D-infinity
    )
    print(f"Downslope distance to stream saved to: {output}")


In [17]:
def calculate_average_flowpath_slope(dem, output):
    """
    Calculate average slope along flowpaths through each cell.
    """
    wbt.average_flowpath_slope(
        dem=dem,
        output=output
    )
    print(f"Average flowpath slope saved to: {output}")


In [18]:
def calculate_average_upslope_flowpath_length(dem, output):
    """
    Calculate average length of upslope flowpaths.
    """
    wbt.average_upslope_flowpath_length(
        dem=dem,
        output=output
    )
    print(f"Average upslope flowpath length saved to: {output}")
